In [1]:
import pandas as pd
import os

# Lista de archivos para combinar
archivos = ['salida_GRCh38.nr_duplications.tsv', 'salida_GRCh38.nr_insertion.tsv', 'salida_GRCh38.nr_deletions_big.tsv']
# Suponiendo que 'archivo3.tsv' es el que no tiene la columna 'gene'
columnas_comunes = ['CHROM', 'POS', 'END', 'method', 'variant_type', 'variant', 'DB']

# Cargar todos los archivos
dataframes = []
for i, archivo in enumerate(archivos):
    if i < 2:  # Los primeros dos archivos tienen todas las columnas
        df = pd.read_csv(archivo, sep='\t', usecols=columnas_comunes + ['gene'])
    else:  # El tercer archivo no tiene la columna 'gene'
        df = pd.read_csv(archivo, sep='\t', usecols=columnas_comunes)
        df['gene'] = pd.NA  # Añadir columna 'gene' con valores NaN
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

# Seleccionar una muestra aleatoria del DataFrame
df_combinado = df_combinado.sample(frac=0.2, random_state=1)  # Ajusta frac según tus necesidades

df_combinado.rename(columns={'variant': 'dbvar_id'}, inplace=True)

# Paso 1: Crear un identificador base
df_combinado['ID_base'] = 'VARHS' + (df_combinado.index + 1).astype(str)

# Paso 2: Identificar y manejar duplicados
# Crear un contador temporal para cada ID_base
df_combinado['temp_count'] = df_combinado.groupby('ID_base').cumcount() + 1

# Solo añadir el sufijo si hay más de una ocurrencia
df_combinado['ID_merged'] = df_combinado.apply(lambda x: f"{x['ID_base']}_{x['temp_count']}" if x['temp_count'] > 1 else x['ID_base'], axis=1)

# Eliminar la columna temporal usada para contar
df_combinado.drop(['temp_count'], axis=1, inplace=True)

# Reordenar columnas para que ID_merged sea la primera
columnas = ['ID_merged'] + [col for col in df_combinado.columns if col != 'ID_merged']
df_combinado = df_combinado[columnas]

# Guardar el dataframe combinado en un nuevo archivo TSV
df_combinado.to_csv('archivo_combinado.tsv', sep='\t', index=False)

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_19836\1450395313.py:15: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep='\t', usecols=columnas_comunes)


In [2]:
# Suponiendo que df_combinado es tu primera base de datos
last_index_1 = df_combinado.index[-1] + 1  # +1 porque los índices comienzan en 0 pero tú estás añadiendo 1 al índice para los IDs
print(last_index_1)


2153955


In [3]:
print(df_combinado.columns.tolist())

['ID_merged', 'CHROM', 'POS', 'END', 'method', 'variant_type', 'dbvar_id', 'gene', 'DB', 'ID_base']


In [4]:
uri_mapping = {
    1: 'NC_000001.11',
    2: 'NC_000002.12',
    3: 'NC_000003.12',
    4: 'NC_000004.12',
    5: 'NC_000005.10',
    6: 'NC_000006.12',
    7: 'NC_000007.14',
    8: 'NC_000008.11',
    9: 'NC_000009.12',
    10: 'NC_000010.11',
    11: 'NC_000011.10',
    12: 'NC_000012.12',
    13: 'NC_000013.11',
    14: 'NC_000014.9',
    15: 'NC_000015.10',
    16: 'NC_000016.10',
    17: 'NC_000017.11',
    18: 'NC_000018.10',
    19: 'NC_000019.10',
    20: 'NC_000020.11',
    21: 'NC_000021.9',
    22: 'NC_000022.11',
    'X': 'NC_000023.11',
    'Y': 'NC_000024.10',
    'mt': 'NC_012920.1'
}

In [5]:
descrip_alt_label_mapping = {
    1: 'NC_000001.11:g.',
    2: 'NC_000002.12:g.',
    3: 'NC_000003.12:g.',
    4: 'NC_000004.12:g.',
    5: 'NC_000005.10:g.',
    6: 'NC_000006.12:g.',
    7: 'NC_000007.14:g.',
    8: 'NC_000008.11:g.',
    9: 'NC_000009.12:g.',
    10: 'NC_000010.11:g.',
    11: 'NC_000011.10:g.',
    12: 'NC_000012.12:g.',
    13: 'NC_000013.11:g.',
    14: 'NC_000014.9:g.',
    15: 'NC_000015.10:g.',
    16: 'NC_000016.10:g.',
    17: 'NC_000017.11:g.',
    18: 'NC_000018.10:g.',
    19: 'NC_000019.10:g.',
    20: 'NC_000020.11:g.',
    21: 'NC_000021.9:g.',
    22: 'NC_000022.11:g.',
    'X': 'NC_000023.11:g.',
    'Y': 'NC_000024.10:g.',
    'mt': 'NC_012920.1:g.'
}

In [6]:
method_mapping = {
    "BAC_aCGH": ["NCIT_C116157"],
    "Curated": ["OBI_0302840"],
    "MLPA": ["NCIT_C116161"],
    "Oligo_aCGH": ["NCIT_C116156"],
    "Optical_mapping": ["NCIT_C116162"],
    "ROMA": ["NCIT_C40211"],
    "SNP_array": [ "NCIT_C40211"], # "http://www.ebi.ac.uk/efo/EFO_0002703",
    # "Sequencing": ["http://www.ebi.ac.uk/efo/EFO_0002693", "http://www.bioassayontology.org/bao#BAO_0040026"],
    "qPCR": [
        "OBI_0000893",
        "ECO_0000231",
        # "http://www.bioassayontology.org/bao#BAO_0003031",
        "NCIT_C51962",
        "MI_1195"
    ]
}

In [7]:
method_mapping_bao = {
    "Sequencing": "BAO_0040026",
    "qPCR": "BAO_0003031"
}

In [8]:
method_mapping_efo = {

    "SNP_array": "EFO_0002703",
    "Sequencing": "EFO_0002693"

}

In [9]:
    # "Merging": ["None"],#invent
    # "Multiple": ["None"],#invent
    # "Not_provided": ["None"],#invent

In [10]:
# Aplicar el mapeo utilizando el diccionario
df_combinado['method_uri'] = df_combinado['method'].map(method_mapping)

# Expandir las listas en la columna 'method_uri' a múltiples filas
df_combinado = df_combinado.explode('method_uri')

In [11]:
# Aplicar el mapeo utilizando el diccionario method_mapping_efo
df_combinado['method_uri_efo'] = df_combinado['method'].map(method_mapping_efo)

In [12]:
# Aplicar el mapeo utilizando el diccionario method_mapping_bao
df_combinado['method_uri_bao'] = df_combinado['method'].map(method_mapping_bao)

In [13]:
genes_mapping = {
    'WT1': '7490',
    'TRDN': '10345',
    'NF2': '4771',
    'RYR2': '6262',
    'SCN5A': '6331',
    'STK11': '6794',
    'BRCA2': '675',
    'TMEM43': '79188',
    'COL3A1': '1281',
    'FBN1': '2200',
    'TGFBR2': '7048',
    'PMS2': '5395',
    'GAA': '2548',
    'CASQ2': '845',
    'TTN': '7273',
    'SDHC': '6391',
    'MSH2': '4436',
    'TMEM127': '55654',
    'BTD': '686',
    'MLH1': '4292',
    'APC': '324',
    'DSP': '1832',
    'FLNC': '2318',
    'KCNH2': '3757',
    'PRKAG2': '51422',
    'TGFBR1': '7046',
    'TSC1': '7248',
    'BMPR1A': '657',
    'ACTA2': '59',
    'KCNQ1': '3784',
    'MYBPC3': '4607',
    'SDHAF2': '54949',
    'SDHD': '6392',
    'MYL2': '4633',
    'RB1': '5925',
    'ATP7B': '540',
    'MAX': '4149',
    'ACTC1': '70',
    'SMAD3': '4088',
    'TSC2': '7249',
    'MYH11': '4629',
    'PALB2': '79728',
    'BRCA1': '672',
    'SMAD4': '4089',
    'LDLR': '3949',
    'RYR1': '6261',
    'TNNI3': '7137',
    'OTC': '5009',
    'SDHB': '6390',
    'MUTYH': '4595',
    'RPE65': '6121',
    'LMNA': '4000',
    'APOB': '338',
    'MSH6': '2956',
    'ENG': '2022',
    'RET': '5979',
    'PTEN': '5728',
    'MEN1': '4221',
    'PKP2': '5318',
    'HNF1A': '6927',
    'TPM1': '7168',
    'TP53': '7157',
    'DSG2': '1829'

}

In [14]:
variant_type_mapping = {'duplication':"SO_1000035",
'copy_number_gain': "SO_0001742",
'copy_number_variation': "SO_0001019",
'herv_insertion': "SO_0002187",
'alu_insertion': "SO_0002063",
'insertion': "SO_0000667",
'sva_insertion': "SO_0002065",
'line1_insertion': "SO_0002064",
'deletion': "SO_0000159",
'copy_number_loss': "SO_0001743",
'alu_deletion': "SO_0002070",
'herv_deletion': "SO_0002067",
'line1_deletion': "SO_0002069",
'sva_deletion': "SO_0002068",
}

In [15]:
new_uri_mapping = {} # El objetivo es acaparar tanto unas URIs de los CHROM,  es que parece que pueden ser str o valores enteros

for k, v in uri_mapping.items():
    if isinstance(k, int):  # Si la clave es entera, añadir tanto la versión entera como la versión string
        new_uri_mapping[k] = v
        new_uri_mapping[str(k)] = v
    else:  # Si la clave ya es un string, solo añadir esa clave
        new_uri_mapping[k] = v

#print(new_uri_mapping)

In [16]:
# Crear una nueva columna en el DataFrame mapeando los valores
df_combinado['CHROM_uri'] = df_combinado['CHROM'].map(new_uri_mapping)

In [17]:
# Supongamos que tu dataframe se llama df y tu diccionario se llama descrip_alt_label_mapping
df_combinado['desc'] = df_combinado['CHROM'].map(descrip_alt_label_mapping)

In [18]:
df_expanded = df_combinado.assign(method_uri=df_combinado['method_uri'].str.split(',')).explode('method_uri')

In [19]:
df_expanded['variant_type_uri'] = df_expanded['variant_type'].map(variant_type_mapping)

In [20]:
# Dividir la columna 'method' en listas donde cada método es un elemento separado
df_expanded['method'] = df_expanded['method'].str.split(';')

# Usar explode para crear una nueva fila por cada elemento de la lista en la columna 'method'
df_exploded_method = df_expanded.explode('method')

In [21]:
# Asegúrate de que la columna 'gene' existe en tu DataFrame
if 'gene' in df_exploded_method.columns:
    # Usa map() para crear la nueva columna 'gene_uri' basada en el diccionario genes_mapping
    df_exploded_method['gene_uri'] = df_exploded_method['gene'].map(genes_mapping)

    # Muestra el DataFrame resultante
    print(df_exploded_method)
else:
    print("La columna 'gene' no existe en el DataFrame.")

            ID_merged CHROM        POS        END      method variant_type  \
3564353  VARHS3564354     5  150823600  150843707     Curated     deletion   
3564353  VARHS3564354     5  150823600  150843707     Merging     deletion   
3564353  VARHS3564354     5  150823600  150843707  Sequencing     deletion   
6869836  VARHS6869837     8  113922162  113922454     Curated     deletion   
6869836  VARHS6869837     8  113922162  113922454     Merging     deletion   
...               ...   ...        ...        ...         ...          ...   
750383    VARHS750384     3  135976981  135982612  Sequencing     deletion   
3872092  VARHS3872093     6   14745051   14745458     Curated     deletion   
3872092  VARHS3872093     6   14745051   14745458     Merging     deletion   
3872092  VARHS3872093     6   14745051   14745458  Sequencing     deletion   
2153954  VARHS2153955     4  136150647  136155884  Sequencing     deletion   

             dbvar_id gene     DB       ID_base method_uri meth

In [22]:
if 'variant_type' in df_exploded_method.columns:
    # Divide los valores de 'variant_type' antes de explotar
    df_exploded_method['variant_type'] = df_exploded_method['variant_type'].str.split(';')
    df_exploded_variant = df_exploded_method.explode('variant_type')

    # Ahora, reemplaza los valores NaN por 'Unknown'
    df_exploded_variant['variant_type'] = df_exploded_variant['variant_type'].fillna('Unknown')

    # Muestra el DataFrame resultante
    print(df_exploded_variant)
else:
    print("La columna 'variant_type' no existe en el DataFrame.")

            ID_merged CHROM        POS        END      method variant_type  \
3564353  VARHS3564354     5  150823600  150843707     Curated     deletion   
3564353  VARHS3564354     5  150823600  150843707     Merging     deletion   
3564353  VARHS3564354     5  150823600  150843707  Sequencing     deletion   
6869836  VARHS6869837     8  113922162  113922454     Curated     deletion   
6869836  VARHS6869837     8  113922162  113922454     Merging     deletion   
...               ...   ...        ...        ...         ...          ...   
750383    VARHS750384     3  135976981  135982612  Sequencing     deletion   
3872092  VARHS3872093     6   14745051   14745458     Curated     deletion   
3872092  VARHS3872093     6   14745051   14745458     Merging     deletion   
3872092  VARHS3872093     6   14745051   14745458  Sequencing     deletion   
2153954  VARHS2153955     4  136150647  136155884  Sequencing     deletion   

             dbvar_id gene     DB       ID_base method_uri meth

In [23]:
specific_row = df_exploded_variant.iloc[2000]
print(specific_row)

ID_merged              VARHS8506102
CHROM                            15
POS                        42111518
END                        42115486
method                      Merging
variant_type               deletion
dbvar_id                essv5044848
gene                            NaN
DB                            dbVar
ID_base                VARHS8506102
method_uri                      NaN
method_uri_efo                  NaN
method_uri_bao                  NaN
CHROM_uri              NC_000015.10
desc                NC_000015.10:g.
variant_type_uri                NaN
gene_uri                        NaN
Name: 8506101, dtype: object


In [24]:
print(df_exploded_variant.head(11))

              ID_merged CHROM        POS        END      method  \
3564353    VARHS3564354     5  150823600  150843707     Curated   
3564353    VARHS3564354     5  150823600  150843707     Merging   
3564353    VARHS3564354     5  150823600  150843707  Sequencing   
6869836    VARHS6869837     8  113922162  113922454     Curated   
6869836    VARHS6869837     8  113922162  113922454     Merging   
6869836    VARHS6869837     8  113922162  113922454  Sequencing   
11811906  VARHS11811907    22   49103537   49106399  Sequencing   
5488514    VARHS5488515     7   67512642   67523698  Sequencing   
11658025  VARHS11658026    22   28699835   28712022  Sequencing   
6849735    VARHS6849736     8  111281828  111284913     Curated   
6849735    VARHS6849736     8  111281828  111284913     Merging   

              variant_type      dbvar_id gene     DB        ID_base  \
3564353           deletion  essv19806087  NaN  dbVar   VARHS3564354   
3564353           deletion  essv19806087  NaN  dbVar 

In [25]:
num_rows = df.shape[0]
print("Número de filas en el DataFrame:", num_rows)

Número de filas en el DataFrame: 11831671


In [26]:
num_rows = len(df)
print("Número de filas en el DataFrame:", num_rows)

Número de filas en el DataFrame: 11831671


In [27]:
# Como tenemos valores NaN voy a ver el contenido de las lineas a ver que se debe y vemos si tienen algun valor distinto a NaN
if 'method_uri_bao' in df_exploded_variant.columns:
    # Obtener los valores únicos en la columna 'SCIDN'
    unique_values = df_exploded_variant['method_uri_bao'].dropna().unique()
    print("Valores únicos en la columna 'method_uri_bao':")
    print(unique_values)
else:
    print("La columna 'method_uri_bao' no existe en el DataFrame.") 

Valores únicos en la columna 'method_uri_bao':
['BAO_0040026']


In [28]:
# Define la ruta y el nombre del archivo TSV de salida
archivo_salida_tsv = 'salida_dbVar.tsv'
carpeta_salida = 'Salidas'
archivo_salida_tsv = os.path.join(carpeta_salida, archivo_salida_tsv)
# Guardar el DataFrame en un archivo TSV
df_exploded_variant.to_csv(archivo_salida_tsv, sep='\t', index=False)

print(f"DataFrame guardado con éxito en '{archivo_salida_tsv}'")

DataFrame guardado con éxito en 'Salidas\salida_dbVar.tsv'


In [29]:
solo_tres_filas = df_exploded_variant.head(1500)

# Guardar las primeras tres filas en un archivo CSV
solo_tres_filas.to_csv('primeras_tripletas_dbVar.tsv', index=False)  # index=False para no incluir el índice del DataFrame en el archivo

In [30]:
'''Consultas'''

'Consultas'

In [31]:
# Agrupa por 'variant_type' y selecciona los primeros 5 registros de cada grupo
consulta_1 = df_exploded_variant.groupby('variant_type').head(5)
print(consulta_1)

            ID_merged CHROM        POS        END      method  \
3564353  VARHS3564354     5  150823600  150843707     Curated   
3564353  VARHS3564354     5  150823600  150843707     Merging   
3564353  VARHS3564354     5  150823600  150843707  Sequencing   
6869836  VARHS6869837     8  113922162  113922454     Curated   
6869836  VARHS6869837     8  113922162  113922454     Merging   
...               ...   ...        ...        ...         ...   
4348        VARHS4349     2   47582865   47582865     Merging   
4296        VARHS4297     2   47582865   47582865     Merging   
7618        VARHS7619    11   32412717   32412717  Sequencing   
9281        VARHS9282    13   48432363   48432363     Merging   
510          VARHS511     6  123433713  123437152     Merging   

             variant_type      dbvar_id  gene     DB       ID_base method_uri  \
3564353          deletion  essv19806087   NaN  dbVar  VARHS3564354        NaN   
3564353          deletion  essv19806087   NaN  dbVar  VAR

In [32]:
# Guardar el DataFrame en un archivo CSV
consulta_1.to_csv('consulta_1_dbVar.tsv', index=False)

In [33]:
consulta_2 = df_exploded_variant[df_exploded_variant['variant_type'] == 'deletion']

In [34]:
# Guarda el DataFrame filtrado en un archivo CSV
consulta_2.to_csv('consulta_2_dbVar.tsv', index=False)

In [35]:
# Filtra el DataFrame para obtener solo los registros donde gene es 'BRCA1'
consulta_3 = df_exploded_variant[df_exploded_variant['gene'] == 'ACTA2']
print(consulta_3)

      ID_merged CHROM       POS       END      method   variant_type  \
7608  VARHS7609    10  88980189  88980189     Merging  alu_insertion   
7601  VARHS7602    10  88980189  88980189     Merging  alu_insertion   
7572  VARHS7573    10  88980172  88980172  Sequencing  alu_insertion   
7591  VARHS7592    10  88980189  88980189     Merging  alu_insertion   
7580  VARHS7581    10  88980189  88980189     Merging  alu_insertion   
7588  VARHS7589    10  88980189  88980189     Merging  alu_insertion   
7597  VARHS7598    10  88980189  88980189     Merging  alu_insertion   

          dbvar_id   gene     DB    ID_base method_uri method_uri_efo  \
7608  essv22087211  ACTA2  dbVar  VARHS7609        NaN            NaN   
7601  essv22087204  ACTA2  dbVar  VARHS7602        NaN            NaN   
7572  nssv15993614  ACTA2  dbVar  VARHS7573        NaN    EFO_0002693   
7591  essv22087194  ACTA2  dbVar  VARHS7592        NaN            NaN   
7580  essv22087183  ACTA2  dbVar  VARHS7581        NaN    

In [36]:
# Guarda el DataFrame filtrado en un archivo CSV llamado 'consulta_3.csv'
consulta_3.to_csv('consulta_3_dbVar.tsv', index=False)

In [37]:
# Filtra el DataFrame para obtener solo los registros donde CHROM es igual a 17
consulta_5 = df_exploded_variant[df_exploded_variant['CHROM'] == 17]
print(consulta_5)

            ID_merged CHROM       POS       END      method variant_type  \
9763874  VARHS9763875    17  73373857  73374687     Curated     deletion   
9763874  VARHS9763875    17  73373857  73374687     Merging     deletion   
9763874  VARHS9763875    17  73373857  73374687  Sequencing     deletion   
9529370  VARHS9529371    17  22322811  22324560     Curated     deletion   
9529370  VARHS9529371    17  22322811  22324560  Sequencing     deletion   
...               ...   ...       ...       ...         ...          ...   
9825625  VARHS9825626    17  81503458  81504445     Curated     deletion   
9825625  VARHS9825626    17  81503458  81504445     Merging     deletion   
9825625  VARHS9825626    17  81503458  81504445  Sequencing     deletion   
9610645  VARHS9610646    17  44353673  44353931  Sequencing     deletion   
9722786  VARHS9722787    17  65497270  65497272  Sequencing     deletion   

             dbvar_id gene     DB       ID_base method_uri method_uri_efo  \
9763874  e

In [38]:
# Guarda el DataFrame filtrado en un archivo CSV llamado 'consulta_5.csv'
consulta_5.to_csv('consulta_5_dbVar.tsv', index=False)